<a href="https://colab.research.google.com/github/Xiaolumang/deakin-ai-challenge2022/blob/main/deakin_Simpsons_VQA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.saving import hdf5_format
import matplotlib.pyplot as plt
import json
import os
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.utils import plot_model
import random
import h5py
from sklearn.metrics import classification_report
import re
import pandas as pd
from collections import Counter
import re

##################
# Verifications:
#################
print('GPU is used.' if len(tf.config.list_physical_devices('GPU')) > 0 else 'GPU is NOT used.')
print("Tensorflow version: " + tf.__version__)


GPU is NOT used.
Tensorflow version: 2.8.2


In [ ]:
#  Validation Images
data_url = "http://206.12.93.90:8080/simpson2022_dataset/simpsons_validation.tar.gz"
zip_path = keras.utils.get_file("simpsons_validation.tar.gz", data_url, extract=True)
imgs_path_val = os.path.dirname(zip_path) + '/simpsons_validation/'

74956800/74941782 [==============================] - 1s 0us/step


In [ ]:

#  Validation Questions
data_url = "http://206.12.93.90:8080/simpson2022_dataset/questions_validation.zip"
zip_path = keras.utils.get_file("questions_validation.zip", data_url,
                                cache_subdir='datasets/questions_validation/', extract=True)
q_val_file = os.path.dirname(zip_path) + '/questions_validation.json'


16384/5995 [=================================================================================] - 0s 0us/step


In [ ]:
import json

In [ ]:
with open('/content/questions_validation.json') as f:
  data = json.loads(f.read())

In [ ]:
pattern_anno = r'"multiple_choice_answer":\s"(\w+)",\s"image_id":\s"(\d+)",\s".+?question_id":\s(\d+)'
with open('/content/questions_validation.json') as f:
  content = f.read()
  anno = re.findall(pattern_anno, content)
  qst = json.loads(content)['questions']

In [ ]:
df_anno = pd.DataFrame(anno,columns=['answer','img_id','question_id'])
df_anno['img_id'] = df_anno['img_id'].apply(lambda x:str(x))
df_anno['question_id'] = df_anno['question_id'].apply(lambda x:str(x))
df_anno['id'] = df_anno['img_id']+df_anno['question_id']

In [ ]:
df_qst = pd.DataFrame(qst)
df_qst.rename(columns={'image_id':'img_id'},inplace=True)
df_qst['img_id'] = df_qst['img_id'].apply(lambda x:str(x))
df_qst['question_id'] = df_qst['question_id'].apply(lambda x:str(x))
df_qst['id'] = df_qst['img_id']+df_qst['question_id']

In [ ]:
df_train = pd.merge(df_anno,df_qst,on='id',how='inner')

In [ ]:
df_train = df_train[['img_id_x','question_id_x','question','answer']]

In [ ]:
df_train.rename(columns ={'img_id_x':'img_id','question_id_x':'question_id'},inplace=True)

In [ ]:
df_train['img_id'] = df_train['img_id'].apply(lambda x:os.path.join(imgs_path_val,'data_val_'+x+'.png'))

In [ ]:
df_train['answer'] = df_train['answer'].apply(lambda x:1 if x =='yes' else 0)

In [ ]:
from collections import Counter

In [ ]:
vocab_size = 15000
c = Counter()
for x in df_train['question'].str.split(' ').values:
  c.update(x)


In [ ]:
vocab = [x[0] for x in c.most_common(vocab_size)]
token_to_num= keras.layers.StringLookup(vocabulary=vocab)
vocab_size = token_to_num.vocabulary_size()

In [ ]:
img_height,img_width=32,32
image_size = (img_height,img_width)

In [ ]:
def encode_single_sample(img_file, q, anno):
  img = tf.io.read_file(img_file)
  img = tf.image.decode_jpeg(img,channels=3)
  img = tf.image.convert_image_dtype(img,tf.float32)
  img = tf.image.resize(img,[img_height,img_width])
  word_splits= tf.strings.split(q, sep=' ')
  q = token_to_num(word_splits)
  return (img,q),anno

In [ ]:
df_train.head(2)

,img_id,question_id,question,answer
0,/root/.keras/datasets/simpsons_validation/data...,35540245,Is there a screen in the picture?,1
1,/root/.keras/datasets/simpsons_validation/data...,40131134,Is the woman wearing a white colored necklace?,0


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((df_train['img_id'],df_train['question'],df_train['answer']))

In [ ]:
len(df_train)

150

In [ ]:
batch_size = 128
train_dataset = train_dataset.map(encode_single_sample,num_parallel_calls=tf.data.AUTOTUNE).padded_batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def build_model(img_size,vocab_size,num_answers):
  img_input = keras.Input(shape=img_size+(3,),name='input_image')
  img = keras.layers.Flatten()(img_input)
  img = keras.layers.Dense(64,activation='relu')(img)

  q_input= keras.Input(shape=(None,),name='input_question')
  q = keras.layers.Embedding(input_dim=vocab_size,output_dim=20)(q_input)
  q = keras.layers.SimpleRNN(64)(q)
  
  mrg = keras.layers.Multiply()([img,q])

  output = keras.layers.Dense(num_answers, activation='softmax',name='output')(mrg)
  vqa_model = keras.Model(inputs=[img_input,q_input],outputs=output)
  vqa_model.compile(keras.optimizers.Adam(lr=0.001),loss=keras.losses.SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])
  return vqa_model


In [ ]:
model = build_model(image_size, vocab_size,2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 input_question (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 flatten (Flatten)              (None, 3072)         0           ['input_image[0][0]']            
                                                                                                  
 embedding (Embedding)          (None, None, 20)     4200        ['input_question[0][0]']         
                                                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_dataset,epochs=13)

Epoch 1/13
2/2 [==============================] - 6s 252ms/step - loss: 0.6991 - accuracy: 0.4267
Epoch 2/13
2/2 [==============================] - 5s 711ms/step - loss: 0.6357 - accuracy: 0.7533
Epoch 3/13
2/2 [==============================] - 5s 485ms/step - loss: 0.5533 - accuracy: 0.7867
Epoch 4/13
2/2 [==============================] - 4s 436ms/step - loss: 0.4425 - accuracy: 0.8200
Epoch 5/13
2/2 [==============================] - 4s 470ms/step - loss: 0.4069 - accuracy: 0.8400
Epoch 6/13
2/2 [==============================] - 2s 239ms/step - loss: 0.3361 - accuracy: 0.8800
Epoch 7/13
2/2 [==============================] - 2s 251ms/step - loss: 0.2878 - accuracy: 0.8733
Epoch 8/13
2/2 [==============================] - 2s 236ms/step - loss: 0.2350 - accuracy: 0.9200
Epoch 9/13
2/2 [==============================] - 2s 249ms/step - loss: 0.1963 - accuracy: 0.9533
Epoch 10/13
2/2 [==============================] - 2s 240ms/step - loss: 0.1552 - accuracy: 0.9733
Epoch 11/13
2/2 [==

In [ ]:
with h5py.File('model.h5', mode='w') as f:
    hdf5_format.save_model_to_hdf5(model, f)
    f.attrs['answers'] = answers
    f.attrs['image_size'] = image_size
    # The vocabulary is stored in two parts.
    f.attrs['vocab1'] = vocab[0:2048]
    f.attrs['vocab2'] = vocab[2048:]